In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import json

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
draft_url = "https://football.fantasysports.yahoo.com/f1/528229/draftresults"
browser.visit(draft_url)
draft_html = browser.html
draft_soup = BeautifulSoup(draft_html, "html.parser")
draft_table = draft_soup.findAll("tbody")[1]
draft_picks = draft_table.findAll("tr")
draft_pick_list = []
for picks in draft_picks:
    pick = picks.findAll("td")
    name = pick[1].text.split(" (")[0]
    team = pick[1].text.split(" (")[1].split(" - ")[0]
    pos = pick[1].text.split(" (")[1].split(" - ")[1].split(")")[0]
    draft_pick_dict = {
        "name": name,
        "NFLTeam": team,
        "Pos": pos,
        "cost": int(pick[2].text[1:]),
        "team": pick[3].text
    }
    draft_pick_list.append(draft_pick_dict)
for player in draft_pick_list:
    if player['name'] == "Todd Gurley II":
        player["cost"] = 40
    if player["name"] == "Dalvin Cook":
        player["cost"] = 64

In [3]:
draft_df = pd.DataFrame(draft_pick_list)
draft_df['keeper'] = ""
draft_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper
0,Dal,QB,6,Dak Prescott,Naggers w/ Attitude,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,
2,NE,TE,64,Rob Gronkowski,crushiNG RON,
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,
5,NYG,RB,74,Saquon Barkley,Bye Week,
6,Buf,RB,29,LeSean McCoy,Nags,
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,
9,Cin,WR,67,A.J. Green,Naggers w/ Attitude,


In [4]:
keeper_df = pd.read_csv('keeper.csv')
keeper_df.head(15)

,player,duration
0,Dalvin Cook,B
1,Todd Gurley II,AB
2,Adam Thielen,B
3,DeAndre Hopkins,B
4,Melvin Gordon III,AB
5,Davante Adams,AB
6,Kenyan Drake,B
7,Golden Tate,AB
8,Corey Clement,B
9,Devonta Freeman,AB


In [5]:
for i, d in draft_df.iterrows():
    for index, keeper in keeper_df.iterrows():
        if d['name'] == keeper['player']:
            draft_df.at[i,'keeper'] = keeper['duration']
            
draft_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper
0,Dal,QB,6,Dak Prescott,Naggers w/ Attitude,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,
2,NE,TE,64,Rob Gronkowski,crushiNG RON,
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,
5,NYG,RB,74,Saquon Barkley,Bye Week,
6,Buf,RB,29,LeSean McCoy,Nags,
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,
9,Cin,WR,67,A.J. Green,Naggers w/ Attitude,


In [6]:
transaction_pages = np.arange(5)*25
transaction_list = []
for page in transaction_pages:
    try:
        transaction_url = 'https://football.fantasysports.yahoo.com/f1/528229/transactions?transactionsfilter=all&count=' + str(page)
        executable_path = {'executable_path': 'chromedriver.exe'}
        browser = Browser('chrome', **executable_path, headless = True)
        browser.visit(transaction_url)
        transaction_html = browser.html
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("tbody")[1]
        transactions = transaction_table.findAll("tr")
        for t in transactions:
            transaction = t.findAll('td')
            try:
                transaction_info = transaction[1].text.splitlines()
                team_info = transaction[2].text.splitlines()
                try:
                    transaction_info.remove("Q")
                except:
                    pass
                try:
                    transaction_info.remove("D")
                except:
                    pass
                try:
                    transaction_info.remove("O")
                except:
                    pass
                try:
                    transaction_info.remove("NA")
                except:
                    pass
                if transaction_info[3].strip()[0] == "$":
                    price = int(transaction_info[3].strip().split("  ")[0][1:])
                else:
                    price = int(0)
                transaction_dict = {
                    "add": transaction_info[1].strip(),
                    "nfl_team": transaction_info[2].strip().split(" - ")[0],
                    "position": transaction_info[2].strip().split(" - ")[1],
                    "price":price,
                    "drop":transaction_info[4].strip(),
                    "ffteam":team_info[3].strip()
                }
                transaction_list.append(transaction_dict)
            except:
                pass
    except:
        pass

In [7]:
transaction_df = pd.DataFrame(transaction_list)
transaction_df = transaction_df.reindex(index=transaction_df.index[::-1]).reset_index(drop=True)
transaction_df = transaction_df.drop([48, 49]).reset_index(drop=True)
transaction_df.head(15)

,add,drop,ffteam,nfl_team,position,price
0,James White,Pittsburgh,Naggers w/ Attitude,NE,RB,2
1,Mason Crosby,Baker Mayfield,2 Gurley's 1 Cup,GB,K,0
2,John Ross,Dez Bryant,crushiNG RON,Cin,WR,0
3,Austin Hooper,Charles Clay,crushiNG RON,Atl,TE,0
4,Graham Gano,Donte Moncrief,Kamara Sutra,Car,K,0
5,Allen Hurns,LeGarrette Blount,K-K-Dai,Dal,WR,0
6,Quincy Enunwa,Denver,K-K-Dai,NYJ,WR,0
7,Jaylen Samuels,Austin Hooper,crushiNG RON,Pit,"RB,TE",0
8,Geronimo Allison,Dede Westbrook,K-K-Dai,GB,WR,0
9,Dede Westbrook,Matt Prater,G,Jax,WR,0


In [8]:
for x, t in transaction_df.iterrows():
    for i, d in draft_df.iterrows():
        if d['name'] == t['drop']:
            draft_df.at[i, 'name'] = t['add']
            draft_df.at[i, 'NFLTeam'] = t['nfl_team']
            draft_df.at[i, 'Pos'] = t['position']
            draft_df.at[i, 'cost'] = t['price']
            draft_df.at[i, 'keeper'] = ""
roster_df = draft_df
roster_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper
0,Dal,QB,6,Dak Prescott,Naggers w/ Attitude,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,
2,NE,TE,64,Rob Gronkowski,crushiNG RON,
3,Jax,RB,60,Leonard Fournette,Kamara Sutra,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,
5,NYG,RB,74,Saquon Barkley,Bye Week,
6,Buf,RB,29,LeSean McCoy,Nags,
7,LAC,QB,11,Philip Rivers,Thx Ron and Le'Veon,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,
9,Cin,WR,67,A.J. Green,Naggers w/ Attitude,


In [9]:
ff_team_names = roster_df.team.unique()
print(ff_team_names)

['Naggers w/ Attitude' "Chi ShingT's Team" 'crushiNG RON' 'Kamara Sutra'
 'Cry me a Philip' 'Bye Week' 'Nags' "Thx Ron and Le'Veon"
 "2 Gurley's 1 Cup" 'FirstRoundFlops' 'K-K-Dai' 'G']


In [10]:
for ff in ff_team_names:
    print(roster_df.loc[roster_df['team'] == ff])
    print("------------------------------------------------------------------------------------")

    NFLTeam  Pos  cost              name                 team keeper
0       Dal   QB     6      Dak Prescott  Naggers w/ Attitude       
9       Cin   WR    67        A.J. Green  Naggers w/ Attitude       
23      LAR   WR    39     Brandin Cooks  Naggers w/ Attitude       
24       TB   WR     2    DeSean Jackson  Naggers w/ Attitude       
28      Phi   TE    49         Zach Ertz  Naggers w/ Attitude       
58      LAR  DEF    11       Los Angeles  Naggers w/ Attitude       
80      Sea   RB    13     Rashaad Penny  Naggers w/ Attitude       
84      NYJ   RB     7      Bilal Powell  Naggers w/ Attitude       
94      Bal    K     4     Justin Tucker  Naggers w/ Attitude       
108     Car   TE     0        Greg Olsen  Naggers w/ Attitude       
120      NE   RB     2       James White  Naggers w/ Attitude       
129      GB   WR     1      Randall Cobb  Naggers w/ Attitude       
177      NO   WR    18    Michael Thomas  Naggers w/ Attitude     AB
178     Min   QB    22      Kirk C